In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [17]:
df=pd.read_csv("/kaggle/input/playground-series-s4e1/train.csv")
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [18]:
# Identify columns with data type 'object'
df=df.drop(['Surname'],axis=1)

object_columns = df.select_dtypes(include=['object']).columns

# Create an instance of the OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the identified columns
encoded_data = encoder.fit_transform(df[object_columns]).toarray()

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(object_columns))

# Concatenate the original DataFrame with the encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Drop the original 'object' columns if needed
df = df.drop(object_columns, axis=1)

# Print the final DataFrame with the encoded columns
print(df)

            id  CustomerId  CreditScore   Age  Tenure    Balance  \
0            0    15674932          668  33.0       3       0.00   
1            1    15749177          627  33.0       1       0.00   
2            2    15694510          678  40.0      10       0.00   
3            3    15741417          581  34.0       2  148882.54   
4            4    15766172          716  33.0       5       0.00   
...        ...         ...          ...   ...     ...        ...   
165029  165029    15667085          667  33.0       2       0.00   
165030  165030    15665521          792  35.0       3       0.00   
165031  165031    15664752          565  31.0       5       0.00   
165032  165032    15689614          554  30.0       7  161533.00   
165033  165033    15732798          850  31.0       1       0.00   

        NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  Exited  \
0                   2        1.0             0.0        181449.97       0   
1                   2        

In [19]:
df=df.drop(["id",'CustomerId'],axis=1)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,668,33.0,3,0.00,2,1.0,0.0,181449.97,0,1.0,0.0,0.0,0.0,1.0
1,627,33.0,1,0.00,2,1.0,1.0,49503.50,0,1.0,0.0,0.0,0.0,1.0
2,678,40.0,10,0.00,2,1.0,0.0,184866.69,0,1.0,0.0,0.0,0.0,1.0
3,581,34.0,2,148882.54,1,1.0,1.0,84560.88,0,1.0,0.0,0.0,0.0,1.0
4,716,33.0,5,0.00,2,1.0,1.0,15068.83,0,0.0,0.0,1.0,0.0,1.0


In [20]:
y=df["Exited"]
X=df.drop('Exited',axis=1)

In [21]:
print(y)
print(X)

0         0
1         0
2         0
3         0
4         0
         ..
165029    0
165030    0
165031    0
165032    0
165033    1
Name: Exited, Length: 165034, dtype: int64
        CreditScore   Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0               668  33.0       3       0.00              2        1.0   
1               627  33.0       1       0.00              2        1.0   
2               678  40.0      10       0.00              2        1.0   
3               581  34.0       2  148882.54              1        1.0   
4               716  33.0       5       0.00              2        1.0   
...             ...   ...     ...        ...            ...        ...   
165029          667  33.0       2       0.00              1        1.0   
165030          792  35.0       3       0.00              1        0.0   
165031          565  31.0       5       0.00              1        1.0   
165032          554  30.0       7  161533.00              1        0.0   
165033     

In [22]:
X.shape

(165034, 13)

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

nn_model=Sequential([
    Dense(16,activation='linear',input_shape=(X.shape[1],)),
    Dropout(0.1),
    Dense(8, activation='linear'),
    Dropout(0.1),
    Dense(4, activation='linear'),
    Dropout(0.1),
    Dense(1, activation='sigmoid'),
])

In [24]:
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                224       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
 dropout_4 (Dropout)         (None, 8)                 0         
                                                                 
 dense_6 (Dense)             (None, 4)                 36        
                                                                 
 dropout_5 (Dropout)         (None, 4)                 0         
                                                                 
 dense_7 (Dense)             (None, 1)                

In [25]:
from tensorflow.keras.optimizers import Adam
custom_learning_rate = 0.001
custom_beta_1 = 0.9
custom_beta_2 = 0.999
custom_epsilon = 1e-7

optimizer = Adam(learning_rate=custom_learning_rate, beta_1=custom_beta_1, beta_2=custom_beta_2, epsilon=custom_epsilon)

nn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(),'MAE','accuracy'])

In [26]:
# Train the model
nn_model.fit(X, y, epochs=10)

Epoch 1/10
5158/5158 [==============================] - 15s 3ms/step - loss: 634.6542 - auc_1: 0.4982 - MAE: 0.3737 - accuracy: 0.6295
Epoch 2/10
5158/5158 [==============================] - 13s 2ms/step - loss: 4.9576 - auc_1: 0.5035 - MAE: 0.3224 - accuracy: 0.7158
Epoch 3/10
5158/5158 [==============================] - 12s 2ms/step - loss: 0.5386 - auc_1: 0.5352 - MAE: 0.3313 - accuracy: 0.7791
Epoch 4/10
5158/5158 [==============================] - 12s 2ms/step - loss: 0.5242 - auc_1: 0.5417 - MAE: 0.3314 - accuracy: 0.7838
Epoch 5/10
5158/5158 [==============================] - 12s 2ms/step - loss: 0.5152 - auc_1: 0.5609 - MAE: 0.3303 - accuracy: 0.7876
Epoch 6/10
5158/5158 [==============================] - 12s 2ms/step - loss: 0.5129 - auc_1: 0.5713 - MAE: 0.3302 - accuracy: 0.7878
Epoch 7/10
5158/5158 [==============================] - 12s 2ms/step - loss: 0.5112 - auc_1: 0.5812 - MAE: 0.3295 - accuracy: 0.7880
Epoch 8/10
5158/5158 [==============================] - 12s 2ms/ste

In [27]:
X_train,y_train=X,y

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDRegressor  
from xgboost import XGBClassifier

# Define models
lr = LogisticRegression()
sgdr = SGDRegressor() 
xgb = XGBClassifier()

# Define parameter grids
lr_params = {'C': [0.01, 0.1, 0.001]}  
sgdr_params = {'alpha': [0.001, 0.01, 0.1]}
xgb_params = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6, 10],
    'n_estimators': [1000, 10000]
}

# Grid Search Cross Validation
lr_cv = GridSearchCV(lr, lr_params, cv=5, scoring='accuracy') 
sgdr_cv = GridSearchCV(sgdr, sgdr_params, cv=5, scoring='neg_mean_squared_error')
xgb_cv = GridSearchCV(xgb, xgb_params, cv=5, scoring='roc_auc')

# Fit models
lr_cv.fit(X_train, y_train) 
sgdr_cv.fit(X_train, y_train)
xgb_cv.fit(X_train, y_train)

# Best hyperparameters
print('LogisticRegression:', lr_cv.best_params_)  
print('SGDRegressor:', sgdr_cv.best_params_)
print('XGBClassifier:', xgb_cv.best_params_)

In [ ]:
def evaluate(model):
    predictions = model.predict(x_test)
    predictions = predictions.reshape(1, len(predictions))[0]
    predictions = np.array(list(map(lambda p: 1 if p >= 0.5 else 0, predictions)))

    miss_rate = round(np.mean(predictions !=  y_test) * 100 , 2)
    print(f'Miss rate: {miss_rate}%')

    return miss_rate

In [ ]:
models = [nn_model, lr_cv, sgdr_cv, xgb_cv]
miss_rates = []

for model in models:
    miss_rates.append(evaluate(model))

In [ ]:
best_model = models[np.argmin(miss_rates)]

In [ ]:
df_test = pd.read_csv("/kaggle/input/playground-series-s4e1/test.csv")
df_id=pd.DataFrame()

# Set the IDs as the index
df_id.index = df_test['id']
df_test=df_test.drop(["id",'CustomerId','Surname'],axis=1)

object_columns = df_test.select_dtypes(include=['object']).columns

# Fit and transform the identified columns
encoded_data = encoder.fit_transform(df_test[object_columns]).toarray()

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(object_columns))

# Concatenate the original DataFrame with the encoded DataFrame
df_test = pd.concat([df_test, encoded_df], axis=1)

# Drop the original 'object' columns if needed
x_test = df_test.drop(object_columns, axis=1)

# Print the final DataFrame with the encoded columns
print(x_test)

test_pred = best_model.predict(x_test)
[test_pred] = test_pred.reshape(1, len(test_pred))
test_pred = np.round(test_pred, 1)
output = pd.DataFrame({'id': test_data.id, 'Exited': test_pred})
output.to_csv('bank-churn-submission16/1.csv', index=False)

In [ ]:
!pip install lazypredict

In [ ]:
print(X)
print(y)

# Test Data

In [ ]:
df_test=pd.read_csv("/kaggle/input/playground-series-s4e1/test.csv")
df_test.head()

In [ ]:
df_id=pd.DataFrame()

# Set the IDs as the index
df_id.index = df_test['id']
df_test=df_test.drop(["id",'CustomerId','Surname'],axis=1)

object_columns = df_test.select_dtypes(include=['object']).columns

# Fit and transform the identified columns
encoded_data = encoder.fit_transform(df_test[object_columns]).toarray()

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(object_columns))

# Concatenate the original DataFrame with the encoded DataFrame
df_test = pd.concat([df_test, encoded_df], axis=1)

# Drop the original 'object' columns if needed
df_test = df_test.drop(object_columns, axis=1)

# Print the final DataFrame with the encoded columns
print(df_test)

In [ ]:
y_pred=model.predict(df_test)
print(y_pred)

In [ ]:
df_id["Exited"]=pred

In [ ]:
print(df_id.tail())

In [ ]:
df_id.to_csv('test.csv')

In [ ]:
submission = pd.read_csv('/kaggle/input/playground-series-s4e1/sample_submission.csv')
submission.head()

In [ ]:
submission.Exited = y_pred
submission.head(30)

In [ ]:
submission.to_csv("submission.csv", columns = submission.columns, index = False)
submission.head()